In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [30]:
df = pd.read_csv(r"C:\Users\maill\Documents\GitHub\SARA\data\fusion_def_clean.csv", low_memory=False)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [35]:
# Séparation de la variable cible y avant l'encodage
y = df['grav']

X = df.drop('grav', axis=1)


In [36]:
#simplification de la variable cible : tué et bléssé grave -> accident grave
#indemne et blessé léger -> accident non grave

y.replace({'Blessé léger':'non grave', 'Blessé hospitalisé':'grave', 'Indemne':'non grave', 'Tué':'grave'}, inplace=True)

In [37]:
X.drop(['Unnamed: 0','annee_x_x','date','an_nais','age_acc','age_acc_seconds', 'num_acc', 'place', 'locp', 'actp','num_veh','an_naiss','age_acc_an','senc','obs','obsm','choc','manv','permis','col','com','dep'], axis=1, inplace=True)
print(X.columns)


Index(['mois', 'jour', 'lum', 'agg', 'int', 'atm', 'jour_de_la_semaine',
       'heure', 'catr', 'circ', 'nbv', 'vosp', 'prof', 'plan', 'surf', 'infra',
       'situ', 'catv_Label', 'catu', 'sexe', 'trajet', 'etatp', 'secuUn',
       'secuDeux', 'tranches_ages'],
      dtype='object')


In [38]:
# Création de l'ensemble d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [39]:
for col in X_train:
    # Calculer la fréquence de chaque catégorie
    frequency_encoding = X_train[col].value_counts(normalize=True)

    # Remplacer chaque catégorie par sa fréquence
    X_train[col] = X_train[col].map(frequency_encoding)
    
for col in X_test:
    # Calculer la fréquence de chaque catégorie
    frequency_encoding = X_test[col].value_counts(normalize=True)

    # Remplacer chaque catégorie par sa fréquence
    X_test[col] = X_test[col].map(frequency_encoding)

In [40]:
#Utilisation des poids de classe pour l'entrainement du modèle RandomForest
# Calculer la fréquence des classes
class_counts = y_train.value_counts()
class_freq = class_counts / len(y_train)

# Inverser la fréquence pour obtenir les poids
class_weights = 1 / class_freq

# Créer un dictionnaire des poids
weights_dict = class_weights.to_dict()
print(weights_dict)

{'non grave': 1.3036482648224692, 'grave': 4.293284091659998}


In [41]:
# Entrainement du modèle
model = RandomForestClassifier(class_weight=weights_dict, random_state=42)

model.fit(X_train, y_train)

RandomForestClassifier(class_weight={'grave': 4.293284091659998,
                                     'non grave': 1.3036482648224692},
                       random_state=42)

In [42]:
# Evaluation du modèle
y_pred = model.predict(X_test)

print(pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))
print(classification_report(y_test, y_pred))


Classe prédite  grave  non grave
Classe réelle                   
grave           34860      66800
non grave       18943     317953
              precision    recall  f1-score   support

       grave       0.65      0.34      0.45    101660
   non grave       0.83      0.94      0.88    336896

    accuracy                           0.80    438556
   macro avg       0.74      0.64      0.66    438556
weighted avg       0.79      0.80      0.78    438556



In [36]:
probs = model.predict_proba(X_test)
print(probs)

[[0.22 0.39 0.37 0.02]
 [0.52 0.31 0.11 0.06]
 [0.56 0.08 0.05 0.31]
 ...
 [0.01 0.33 0.66 0.  ]
 [0.04 0.44 0.52 0.  ]
 [0.1  0.73 0.16 0.01]]


In [ ]:
# Exportation du modèle

import pickle

with open('random_forest_model.pkl','wb') as file:
    pickle.dump(model, file)